# Uncertainty estimation prototype

For https://github.com/gammapy/gammapy/pull/2255

Show how to compute spectral model error band using

1. Differentials
2. Samples

as suggested in https://github.com/gammapy/gammapy/pull/2255#issuecomment-546118689

We use a spectral model with the values and covariance taken from here:
https://github.com/open-gamma-ray-astro/joint-crab/blob/master/results/fit/fit_veritas.yaml

In [1]:
import numpy as np
import astropy.units as u
from gammapy.modeling.models import LogParabolaSpectralModel
from scipy.optimize.slsqp import approx_jacobian
from functools import partial

In [2]:
model = LogParabolaSpectralModel(
    amplitude=3.76e-11 * u.Unit("cm-2 s-1 TeV-1"),
    reference=1 * u.TeV,
    alpha=2.44,
    beta=0.25,
)
model.parameters.covariance = [
    [1.31e-23, 0, -6.80e-14, 3.04e-13],
    [0, 0, 0, 0],
    [-6.80e-14, 0, 0.00899, 0.00904],
    [3.04e-13, 0, 0.00904, 0.0284],
]

## Differentials

The current `evaluate_error` is based on the Python `uncertainties` package,
which is based on differentials (computed with high precision, using autograd).

Unfortunately it doesn't work for all models, and doesn't support `astropy.units.Quantity`,
so let's code up our own solution. It will be less accurate (using finite differential steps)
and probably also slower, but it will work for any model.

To do the error propagation, we use the common approximation:
https://en.wikipedia.org/wiki/Propagation_of_uncertainty#Non-linear_combinations

- `p` = parameter vector
- `C` = covariance matrix for parameters `p`
- `f` = derived quantity, here `dN/dE(E)`
-  `df/dp` = partial derivative vector, how `f` changes with `p`

``f_err ^ 2 = (df/dp) @ C @ (df/dp)``

In [3]:
def gradient(model, energy, eps=1e-6):
    """Compute gradient
    """
    n = len(model.parameters)
    f = model(energy)
    shape = (n, len(np.atleast_1d(energy)))
    df_dp = np.zeros(shape)
    
    for idx, parameter in enumerate(model.parameters):
        if parameter.frozen:
            continue

        # TODO: is this a good step? Is there a better way?        
        dp = eps * model.parameters.error(idx)
        parameter.value += dp
        df = model(energy) - f
        df_dp[idx] = df.value / dp

        # Reset model to original parameter
        parameter.value -= dp
    
    return df_dp    


def gradient_scipy(model, energy, eps=1e-12):
    x = model.parameters.values
    frozen = np.array([_.frozen for _ in model.parameters])
    
    def func(xk):
        return model.evaluate(energy.to_value("TeV"), *xk)

    grad = approx_jacobian(x=x, func=func, epsilon=eps)
    grad[:, frozen] = 0
    return grad.T

In [4]:
%%timeit
gradient(model, [1, 10, 100] * u.TeV)

979 µs ± 17.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [5]:
%%timeit
gradient_scipy(model, [1, 10, 100] * u.TeV)

307 µs ± 12.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
def _as_scalar(array):
    """Parse array as scalar if possible """
    try:
        return array.item()
    except ValueError:
        return array

def evaluate_error(model, energy):
    """New implementation of model.evaluate_error.
    
    TODO: vectorise to work for array `energy` (and keep scalar working)
    
    TODO: document
    """
    C = model.parameters.covariance
    df_dp = gradient_scipy(model, energy)
    f_cov = np.dot(df_dp.T, np.dot(C, df_dp))
    err = np.sqrt(np.diagonal(f_cov))
    return _as_scalar(err)

In [7]:
model.evaluate_error(1 * u.TeV)[1].value

3.6193922141707712e-12

In [8]:
evaluate_error(model, 1 * u.TeV)

3.619392214170783e-12

In [9]:
model.evaluate_error([0.1, 1, 10, 30, 100, 300] * u.TeV)[1].value

array([2.02268909e-09, 3.61939221e-12, 3.62436537e-14, 1.09450339e-15,
       9.37507775e-18, 5.74653364e-20])

In [10]:
evaluate_error(model, [0.1, 1, 10, 30, 100, 300] * u.TeV)

array([2.02262782e-09, 3.61939221e-12, 3.62442853e-14, 1.09441060e-15,
       9.37521522e-18, 5.74665748e-20])

## Samples

Sample parameters from the covariance matrix,
and compute the distribution of the quantity of interest.

- https://docs.scipy.org/doc/numpy/reference/random/generated/numpy.random.Generator.multivariate_normal.html
- https://docs.astropy.org/en/stable/uncertainty/

This is easy to implement if we do a Python for loop over sampled parameter sets.
Typically one would do 10_000 to get a ~ 1% error.
That's probably pretty slow.

TODO: can we vectorise the evaluation over array energies and array parameters?

In [11]:
def evaluate_error_sample(model, energy, n_samples=100):
    """Error propagation using sampling"""
    raise NotImplementedError

In [12]:
evaluate_error_sample(model, 1 * u.TeV)

NotImplementedError: 